In [2]:
!pip install --upgrade tensorflow

  Cache entry deserialization failed, entry ignored
  Using cached https://files.pythonhosted.org/packages/d3/59/d88fe8c58ffb66aca21d03c0e290cd68327cc133591130c674985e98a482/tensorflow-1.14.0-cp27-cp27mu-manylinux1_x86_64.whl
  Cache entry deserialization failed, entry ignored
  Cache entry deserialization failed, entry ignored
  Using cached https://files.pythonhosted.org/packages/05/d2/f94e68be6b17f46d2c353564da56e6fb89ef09faeeff3313a046cb810ca9/mock-3.0.5-py2.py3-none-any.whl
  Cache entry deserialization failed, entry ignored
    100% |████████████████████████████████| 2.5MB 409kB/s 
  Cache entry deserialization failed, entry ignored
  Cache entry deserialization failed, entry ignored
  Using cached https://files.pythonhosted.org/packages/f4/37/e6a7af1c92c5b68fb427f853b06164b56ea92126bcfd87784334ec5e4d42/tensorboard-1.14.0-py2-none-any.whl
  Using cached https://files.pythonhosted.org/packages/c5/db/e56e6b4bbac7c4a06de1c50de6fe1ef3810018ae11732a50f15f62c7d050/enum34-1.1.6-py2-none

In [10]:
import tensorflow as tf
import tensorflow.contrib.layers as layers
import numpy as np
import data_util
from model_components import task_specific_attention, bidirectional_rnn
import pandas as pd
import _pickle as pickle

In [17]:
class HANClassifierModel():
  """ Implementation of document classification model described in
    `Hierarchical Attention Networks for Document Classification (Yang et al., 2016)`
    (https://www.cs.cmu.edu/~diyiy/docs/naacl16.pdf)"""

  def __init__(self,
               vocab_size,
               embedding_size,
               classes,
               word_cell,
               sentence_cell,
               word_output_size,
               sentence_output_size,
               max_grad_norm,
               dropout_keep_proba,
               is_training=None,
               learning_rate=1e-4,
               device='/cpu:0',
               scope=None):
    self.vocab_size = vocab_size
    self.embedding_size = embedding_size
    self.classes = classes
    self.word_cell = word_cell
    self.word_output_size = word_output_size
    self.sentence_cell = sentence_cell
    self.sentence_output_size = sentence_output_size
    self.max_grad_norm = max_grad_norm
    self.dropout_keep_proba = dropout_keep_proba

    with tf.variable_scope(scope or 'tcm') as scope:
      self.global_step = tf.Variable(0, name='global_step', trainable=False)

      if is_training is not None:
        self.is_training = is_training
      else:
        self.is_training = tf.placeholder(dtype=tf.bool, name='is_training')

      self.sample_weights = tf.placeholder(shape=(None,), dtype=tf.float32, name='sample_weights')

      # [document x sentence x word]
      self.inputs = tf.placeholder(shape=(None, None, None), dtype=tf.int32, name='inputs')

      # [document x sentence]
      self.word_lengths = tf.placeholder(shape=(None, None), dtype=tf.int32, name='word_lengths')

      # [document]
      self.sentence_lengths = tf.placeholder(shape=(None,), dtype=tf.int32, name='sentence_lengths')

      # [document]
      self.labels = tf.placeholder(shape=(None,), dtype=tf.int32, name='labels')

      (self.document_size,
        self.sentence_size,
        self.word_size) = tf.unstack(tf.shape(self.inputs))

      self._init_embedding(scope)

      # embeddings cannot be placed on GPU
      with tf.device(device):
        self._init_body(scope)

    with tf.variable_scope('train'):
      self.cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=self.labels, logits=self.logits)

#       self.loss = tf.reduce_mean(tf.multiply(self.cross_entropy, self.sample_weights))
        
      self.loss = tf.reduce_mean(self.cross_entropy)

      tf.summary.scalar('loss', self.loss)

      self.accuracy = tf.reduce_mean(tf.cast(tf.nn.in_top_k(self.logits, self.labels, 1), tf.float32))
      tf.summary.scalar('accuracy', self.accuracy)

      tvars = tf.trainable_variables()

      grads, global_norm = tf.clip_by_global_norm(
        tf.gradients(self.loss, tvars),
        self.max_grad_norm)
      tf.summary.scalar('global_grad_norm', global_norm)

      opt = tf.train.AdamOptimizer(learning_rate)

      self.train_op = opt.apply_gradients(
        zip(grads, tvars), name='train_op',
        global_step=self.global_step)

      self.summary_op = tf.summary.merge_all()

  def _init_embedding(self, scope):
    with tf.variable_scope(scope):
      with tf.variable_scope("embedding") as scope:
        self.embedding_matrix = tf.get_variable(
          name="embedding_matrix",
          shape=[self.vocab_size, self.embedding_size],
          initializer=layers.xavier_initializer(),
          dtype=tf.float32)
        self.inputs_embedded = tf.nn.embedding_lookup(
          self.embedding_matrix, self.inputs)

  def _init_body(self, scope):
    with tf.variable_scope(scope):

      word_level_inputs = tf.reshape(self.inputs_embedded, [
        self.document_size * self.sentence_size,
        self.word_size,
        self.embedding_size
      ])
      word_level_lengths = tf.reshape(
        self.word_lengths, [self.document_size * self.sentence_size])

      with tf.variable_scope('word') as scope:
        word_encoder_output, _ = bidirectional_rnn(
          self.word_cell, self.word_cell,
          word_level_inputs, word_level_lengths,
          scope=scope)

        with tf.variable_scope('attention') as scope:
          word_level_output = task_specific_attention(
            word_encoder_output,
            self.word_output_size,
            scope=scope)

        with tf.variable_scope('dropout'):
          word_level_output = layers.dropout(
            word_level_output, keep_prob=self.dropout_keep_proba,
            is_training=self.is_training,
          )

      # sentence_level

      sentence_inputs = tf.reshape(
        word_level_output, [self.document_size, self.sentence_size, self.word_output_size])

      with tf.variable_scope('sentence') as scope:
        sentence_encoder_output, _ = bidirectional_rnn(
          self.sentence_cell, self.sentence_cell, sentence_inputs, self.sentence_lengths, scope=scope)

        with tf.variable_scope('attention') as scope:
          sentence_level_output = task_specific_attention(
            sentence_encoder_output, self.sentence_output_size, scope=scope)

        with tf.variable_scope('dropout'):
          sentence_level_output = layers.dropout(
            sentence_level_output, keep_prob=self.dropout_keep_proba,
            is_training=self.is_training,
          )

      with tf.variable_scope('classifier'):
        self.logits = layers.fully_connected(
          sentence_level_output, self.classes, activation_fn=None)

        self.prediction = tf.argmax(self.logits, axis=-1)

  def get_feed_data(self, x, y=None, class_weights=None, is_training=True):
    x_m, doc_sizes, sent_sizes = data_util.batch(x)
    fd = {
      self.inputs: x_m,
      self.sentence_lengths: doc_sizes,
      self.word_lengths: sent_sizes,
    }
    if y is not None:
      fd[self.labels] = y
      if class_weights is not None:
        fd[self.sample_weights] = [class_weights[yy] for yy in y]
      else:
        fd[self.sample_weights] = np.ones(shape=[len(x_m)], dtype=np.float32)
    fd[self.is_training] = is_training
    return fd


In [24]:
try:
    from tensorflow.contrib.rnn import LSTMCell, LSTMStateTuple, GRUCell
except ImportError:
        LSTMCell = tf.nn.rnn_cell.LSTMCell
        LSTMStateTuple = tf.nn.rnn_cell.LSTMStateTuple
        GRUCell = tf.nn.rnn_cell.GRUCell

tf.reset_default_graph()
with tf.Session() as session:
    model = HANClassifierModel(
      vocab_size=10,
      embedding_size=5,
      classes=2,
      word_cell=GRUCell(10),
      sentence_cell=GRUCell(10),
      word_output_size=10,
      sentence_output_size=10,
      max_grad_norm=5.0,
      dropout_keep_proba=0.5,
    )
    session.run(tf.global_variables_initializer())

    fd = {
      model.is_training: False,
      model.inputs: [[
        [5, 4, 1, 0],
        [3, 3, 6, 7],
        [6, 7, 0, 0]
      ],
        [
        [2, 2, 1, 0],
        [3, 3, 6, 7],
        [0, 0, 0, 0]
      ]],
      model.word_lengths: [
        [3, 4, 2],
        [3, 4, 0],
      ],
      model.sentence_lengths: [3, 2],
      model.labels: [0, 1],
    }

#     print(session.run(model.logits, fd))
    step, summaries, loss, accuracy, _ = session.run([
          model.global_step,
          model.summary_op,
          model.loss,
          model.accuracy,
          model.train_op,
      ], fd)
    print(summaries)
    print(loss)
    print(accuracy)

b'\n\x11\n\ntrain/loss\x15\x08\xd4/?\n\x15\n\x0etrain/accuracy\x15\x00\x00\x80?\n\x1d\n\x16train/global_grad_norm\x15iN\x14>'
0.6868291
1.0


In [5]:
def load_data(filename='dataset/dtoc.pkl'):
    pd_dtoc = pickle.load(open(filename, 'rb'))
    return pd_dtoc

In [6]:
df_proc = load_data(filename='/home/jupyter/rich/dtoc_proc.pkl') 

,id,sp_no,ep_no,start_date,diag1,diag2,diag3,diag4,diag5,diag6,...,proc12,spell_time,dest_code,adm_code,age,gender,loe,is_oversea,los,is_dtoc
0,M1355572,62295900,1,2005-02-16 00:00:00,Z349,None,None,None,None,None,...,None,2005-02-16 00:00:00,19,31,18,2,4232,0,4232.0,0.0
1,M581130,62784072,1,2008-04-11 00:00:00,Z349,None,None,None,None,None,...,None,2008-04-11 00:00:00,19,31,28,2,2303,0,2303.0,0.0
2,M1588346,63195747,1,2010-10-05 00:00:00,None,None,None,None,None,None,...,None,2010-10-05 00:00:00,19,31,36,2,1241,0,1241.0,0.0
3,M1587381,63354756,1,2011-10-03 00:00:00,O701,Z370,None,None,None,None,...,None,2011-10-03 00:00:00,19,31,19,2,864,0,864.0,0.0
4,M1564074,63357984,1,2011-10-10 00:00:00,O701,Z370,None,None,None,None,...,None,2011-10-10 00:00:00,19,31,31,2,1774,0,1774.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1195236,5087844,4313925,1,2018-10-31 00:49:00,None,None,None,None,None,None,...,None,2018-10-31 00:49:00,98,21,88,1,30011,1,NaN,0.0
1195237,1253471,4313971,1,2018-10-31 00:59:00,None,None,None,None,None,None,...,None,2018-10-31 00:59:00,98,21,32,2,30011,0,NaN,0.0
1195238,5281325,4313949,1,2018-10-31 01:41:00,None,None,None,None,None,None,...,None,2018-10-31 01:41:00,98,21,13,1,30011,0,NaN,0.0
1195239,3034556,4314014,1,2018-10-31 01:42:00,None,None,None,None,None,None,...,None,2018-10-31 01:42:00,98,21,0,2,30011,0,NaN,0.0
